# Indeks



- [CELL INDEX: 0](#Laster variabel-fanen fra regneark)
...



# Laster variabel-fanen fra regneark
nin3_variabler bør bare leses eller kopieres, ikke endre dataframe 'nin3_variabler' etter denne kodeblokken

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
nin3_variabler = pd.read_excel('inn_data/NiN3.0_Tot_e15_20231123_import_kodebase_NKfix.xlsx', sheet_name='Variabler')
nin3_variabler = nin3_variabler.applymap(lambda x: x.strip() if isinstance(x, str) else x) # removing whitespace
nin3_variabler = nin3_variabler.astype(str) # setting all columns to string

## Variabel CSV

In [ ]:
# hent unike kombinasjoner av kolonnene '3 ABC' og '4 NM'
#variabler = nin3_variabler[['3 ABC', '4 NM']].drop_duplicates().sort_values(by=['3 ABC', '4 NM'])
# above line but also drop nan values
variabler = nin3_variabler[['3 ABC', '4 NM']].drop_duplicates().dropna().sort_values(by=['3 ABC', '4 NM'])
variabler['Variabel_kode'] = variabler[['3 ABC', '4 NM']].apply(lambda x: '-'.join(x), axis=1)

variabler.to_csv('ut_data/variabel.csv', index=False, sep=";")
variabler.head(10)


## Variabelnavn <> Variabel mapping CSV

In [ ]:
#nin3_variabler.columns
vvm = nin3_variabler[nin3_variabler['11 Tr/Kl'] == 'W'][['Langkode', 'Kortkode','3 ABC', '4 NM', '5 kat', '6 FG/EK', '7 Varkode1', '8 VarKode2', 'VarNavn']].drop_duplicates()
vvm = vvm.rename(columns={'5 kat': 'Variabelkategori2', 
                        '6 FG/EK': 'Variabeltype', '7 Varkode1': 'Variabelgruppe', '8 VarKode2': 'Variabelnavn_kode', 'VarNavn': 'Variabelnavn_navn'})
vvm['Variabel_kode'] = vvm[['3 ABC', '4 NM']].apply(lambda x: '-'.join(x), axis=1)
#dupl = vvm['Variabelnavn_kode'].duplicated().any()
vvm.to_csv('ut_data/variabelnavn_variabel_mapping.csv', index=False, sep=";")
#vvm

## Måleskala_enhet_1: Opprett dataframe data SO og SI (fra regneark)
Del 1 av 2

In [ ]:
nin3_variabler_so_si = nin3_variabler[nin3_variabler['11 Tr/Kl'] != 'W']
me_1 = nin3_variabler_so_si[['8 VarKode2', '10 Målesk']].copy()
me_1 = me_1.rename(columns={'10 Målesk': 'Måleskala'})
me_1 = me_1.dropna(subset=['Måleskala'])
me_1['Enhet'] = None
me_1.loc[me_1['Måleskala'] == 'SO', 'Enhet'] = 'VSO'
me_1.loc[me_1['Måleskala'] == 'SI', 'Enhet'] = 'VSI'
me_1['MåleskalaNavn'] = me_1['8 VarKode2'] + '-' + me_1['Måleskala']
me_1 = me_1.drop('8 VarKode2', axis=1)
me_1 = me_1.drop_duplicates()
me_1.head(5)

## Måleskala_enhet_2: fra markdown,  join med Måleskala_enhet_1
Del 2 av 2

In [ ]:
from tabulate import tabulate
from io import StringIO
with open('inn_data/maaleskala_trinn_enhet.md', 'r') as file:
    file_string = file.read()
# Remove leading/trailing white spaces and split the string into lines
lines = [line.strip() for line in file_string.strip().split('\n')]
# Join the lines back together, this time separating them with '\n', and read the result into a DataFrame
me_2 = pd.read_csv(StringIO('\n'.join(lines)), sep='|',engine='python')
# Remove any leading/trailing white spaces from the column names and drop any empty rows
me_2.columns = me_2.columns.str.strip()
me_2 = me_2.dropna(how='all')
me_2 = me_2.drop(['Trinn', 'Trinnverdi'], axis=1)
me_2 = me_2[me_2['Måleskala'].str.strip() != '--'] #removing rows where Måleskala = "--"
me_2 = me_2.rename(columns={'Enhet': 'Everdi'})
me_2['Enhet'] = None
me_2.loc[me_2['Everdi'].str.strip() == 'Prosent', 'Enhet'] = 'P'
me_2.loc[me_2['Everdi'].str.strip() == 'Observert antall', 'Enhet'] = 'OA'
me_2.loc[me_2['Everdi'].str.strip() == 'Tetthet', 'Enhet'] = 'T'
me_2.loc[me_2['Everdi'].str.strip() == 'Binær', 'Enhet'] = 'B'
me_2 = me_2.drop(['Everdi'], axis=1)
# make all rows unique
me_2 = me_2.drop_duplicates()
me_2['Måleskala'] = me_2['Måleskala'].str.strip()
me_2['MåleskalaNavn'] = me_2['Måleskala']
me = pd.concat([me_1, me_2], ignore_index=True)
me.to_csv('ut_data/maaleskala_enhet.csv', index=False, sep=";")
me.head(3)


## Måleskala_trinn, del 1 av 2 (SO & SI)
Leser måleskala sammen med trinn for SO og SI deretter leses måleskala trinn fra MD fil
Del 2 produserer den resulterende filen 'ut_data/maaleskala_trinn.csv'

In [ ]:
mt1 = nin3_variabler[['Kortkode', '8 VarKode2', '10 Målesk', '11 Tr/Kl', 'Trinn/klassebetegnelse']].copy()
mt1 = mt1[mt1['11 Tr/Kl'] != 'W']
mt1 = mt1.dropna(subset=['10 Målesk'])
mt1['MåleskalaNavn'] = mt1['8 VarKode2'].astype(str) + '-' + mt1['10 Målesk'].astype(str).str.strip()
mt1 = mt1.rename(columns={'11 Tr/Kl': 'Trinn', 'Trinn/klassebetegnelse': 'Trinnverdi'})
mt1['Trinn'] = mt1['8 VarKode2'].astype(str) + '_' + mt1['Trinn'].astype(str)
mt1 = mt1.drop(['Kortkode', '8 VarKode2', '10 Målesk'], axis=1)
mt1.head(3)

## Måleskala_trinn, del 2 av 2

In [ ]:
# TODO-Fetch variabelnavnkode and måleskalakode
# hent trinn fra md-fil
# tilpass og concat med del1 dataframe
from tabulate import tabulate
from io import StringIO
with open('inn_data/maaleskala_trinn_enhet.md', 'r') as file:
    file_string = file.read()
# Remove leading/trailing white spaces and split the string into lines
lines = [line.strip() for line in file_string.strip().split('\n')]
# Join the lines back together, this time separating them with '\n', and read the result into a DataFrame
mt2 = pd.read_csv(StringIO('\n'.join(lines)), sep='|',engine='python')
# Remove any leading/trailing white spaces from the column names and drop any empty rows
mt2.columns = mt2.columns.str.strip()
mt2 = mt2.dropna(how='all')

mt2['Trinnverdi'] = mt2['Trinnverdi'].str.replace(' ', '')
mt2['MåleskalaNavn'] = mt2['Måleskala'].str.strip()
mt2 = mt2.drop(['Enhet', 'Måleskala'], axis=1)
mt2['Trinnverdi'] = mt2['Trinnverdi'].fillna(0)
mt2
mt = pd.concat([mt1, mt2], ignore_index=True)
# Strip Trinn for white spaces
mt['Trinn'] = mt['Trinn'].str.strip()
# Drop rows where Trinn = '--'
mt = mt[mt['Trinn'] != '--']
# Set first letter in trinnverdi to upper case
mt.loc[mt['Trinnverdi'].str.len() > 0, 'Trinnverdi'] = mt.loc[mt['Trinnverdi'].str.len() > 0, 'Trinnverdi'].str.replace(';', ':')
mt['Trinnverdi'] = mt['Trinnverdi'].str.capitalize() # capitalize first letter
mt = mt[mt['Trinn'] != 'nan_nan']
mt.to_csv('ut_data/maaleskala_trinn.csv', index=False, sep=";")
display(mt.head(3))
mt.tail(10)

## Variabelnavn <> Måleskala mapping

In [ ]:
import pandas as pd
vn_ms = nin3_variabler[nin3_variabler['11 Tr/Kl'] == 'W'][['Kortkode', '8 VarKode2', '10 Målesk']].drop_duplicates()
#display(vn_ms)
#display(vn_ms[vn_ms['Kortkode'] == 'LM-KI_e'])
vn_ms = vn_ms.dropna(subset=['10 Målesk'])  # drop rows where '10 Målesk' has no value
vn_ms = vn_ms.dropna(subset=['8 VarKode2']) # drop rows where '8 VarKode2' has no value
# create a new dataframe to store the replicated rows
new_rows = []


# iterate over each row in the dataframe
for index, row in vn_ms.iterrows():
    # check if the '10 Målesk' column has multiple values separated by comma
    if ',' in row['10 Målesk']:
        # if yes, split the values and create a new row for each value
        values = row['10 Målesk'].split(',')
        for value in values:
            new_row = row.copy()
            new_row['10 Målesk'] = value.strip()
            new_rows.append(new_row)
    else:
        # if no, just append the original row to the new dataframe
        new_rows.append(row)    

# create a new dataframe from the list of new rows
vn_ms_new = pd.DataFrame(new_rows)

# add Varkode2+"-" in front of [10 Målesk]-value if [10 Målesk] is "SO" or "SI"
mask = (vn_ms_new['10 Målesk'] == 'SO') | (vn_ms_new['10 Målesk'] == 'SI')
vn_ms_new.loc[mask, '10 Målesk'] = vn_ms_new.loc[mask, '8 VarKode2'] + '-' + vn_ms_new.loc[mask, '10 Målesk']
# replace underscores with dash in Kortkode column
vn_ms_new['Kortkode'] = vn_ms_new['Kortkode'].str.replace('_', '-')
# make all rows unique
vn_ms_new = vn_ms_new.drop_duplicates()
vn_ms_new = vn_ms_new.drop(['8 VarKode2'], axis=1)
# display the new dataframe
vn_ms_new.rename(columns={'Kortkode':'Variabelnavn_kortkode', '10 Målesk': 'Måleskala'}, inplace=True)
vn_ms_new.to_csv('ut_data/variabelnavn_maaleskala_mapping.csv', index=False, sep=";")
vn_ms_new.head(3)

## variabelnavn_kode, varkode2, oppslagsfil til inn_data

In [ ]:
import pandas as pd
vnkode_varkode2 = nin3_variabler[nin3_variabler['11 Tr/Kl'] == 'W'][['Kortkode', '8 VarKode2', '10 Målesk']]
#display(vnkode_varkode2)
vnkode_varkode2 = vnkode_varkode2.rename(columns={'8 VarKode2': 'Varkode2_kopi'})
#vnkode_varkode2 = vnkode_varkode2[vnkode_varkode2['10 Målesk'].isin(['SO', 'SI','SO,K'])]
vnkode_varkode2 = vnkode_varkode2[vnkode_varkode2['10 Målesk'].str.contains('SO|SI')]
vnkode_varkode2 = vnkode_varkode2.drop_duplicates()
vnkode_varkode2 = vnkode_varkode2.drop(['10 Målesk'], axis=1)
vnkode_varkode2.to_csv('inn_data/variabelnavnkode_varkode2.csv', index=False, sep=";")
vnkode_varkode2.info()

## Koder, Objekttyper v2.3 (hjelpemetoder for konvertering v3.0 -> v2.3)
--> Function: get_23_koder() -> dict

In [ ]:
import pandas as pd
import sqlite3
import csv
import os
import urllib.parse

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("nin2prod.db")
query = """
    Select distinct KodeName, 'Type' as Klasse from Kode where VersionId in (select id from NiNVersion where Navn ='2.3')
    UNION
    Select distinct KodeName, 'Variabel' as Klasse from VariasjonKode where VersionId in (select id from NiNVersion where Navn ='2.3')
    Order by Klasse"""
df = pd.read_sql_query(query, con)

# Verify that result of SQL query is stored in the dataframe

con.close()
df.to_csv('inn_data/2_3_koder_fra_sqlite_prod.csv', sep=';', index=False)

# Load v2.3 koder to dict
koder23 = {}
with open('inn_data/2_3_koder_fra_sqlite_prod.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';', quotechar='"')
    koder = []
    for row in reader:
        if ',' in row['KodeName']:
            # do something if the string contains a comma
            kode_list = row['KodeName'].split(',')
            for kode in kode_list:
                lookupkode = kode.lower().strip() #lager lookupkolonne som lowercase siden kode fra excel er ..
                # ..ukorrekt i bruk av  i bruk små og store bokstaver
                if lookupkode.startswith("na "):
                    lookupkode = lookupkode.replace("na ", "")
                koder23[lookupkode.strip()] = {"KondeName":row['KodeName'].strip(), "Klasse":row['Klasse']}
                #koder23[kode.strip()] = row['Klasse']
        else:
            # do something else if the string does not contain a comma
            lookupkode = row['KodeName'].lower().strip() #lager lookupkolonne som lowercase siden kode fra excel er ..
                # ..ukorrekt i bruk av  i bruk små og store bokstaver
            if lookupkode.startswith("na "):
                lookupkode = lookupkode.replace("na ", "")
            koder23[lookupkode.strip()] = {"KodeName":row['KodeName'].strip(), "Klasse":row['Klasse']}

# Write koder23 to a text file in the tmp directory
with open(os.path.join('tmp', 'koder23.txt'), 'w') as f:
    output = []
    for key, value in koder23.items():
        output.append(f"{key}: {value}\n")
    f.write("".join(output))


# TODO: Hvis koden ikke får tregg forsøk oppslag med å bytt "-" med 
def create_v23_variabel_url(kode):
    if(kode=="V12"):
        print("V12")
    kode = kode.lower()
    var_url = 'https://nin-kode-api.artsdatabanken.no/v2.3/variasjon/hentkode/'
    type_url = 'https://nin-kode-api.artsdatabanken.no/v2.3/koder/hentkode/'
    result = {}
    if koder23.get(kode):
        kodeentry = koder23.get(kode)
        if kodeentry["Klasse"] == 'Variabel':
            result["kode23"]=kodeentry['KodeName']
            result["url"]=f"{var_url}{urllib.parse.quote(kodeentry['KodeName'])}" 
            return result
        elif kodeentry["Klasse"] == 'Type':
            result["kode23"]=kodeentry['KodeName']
            result["url"]=f"{type_url}{urllib.parse.quote(kodeentry['KodeName'])}" 
            return result
        else:
            return {}
    else:
        return {}
    
def make_list_nin2kode(nin2kode):
    reslist = []
    n2list = nin2kode.split(',')
    first = n2list[0]
    for n2 in n2list:
        if n2.isdigit():
            reslist.append(first.split('-')[0]+'-'+n2)
        else:
            reslist.append(n2)
    return reslist

# Variabelnavn, konvertering (Under arbeid, kjørte på siste forsøk, men noen ikke funnet feil)
sample:

Fant ikke url for NiN2kode:KY

Fant ikke url for NiN2kode:DM

Fant ikke url for NiN2kode:KLG-IYK3

Fant ikke url for NiN2kode:KLG-4

... (m.fl.)

In [ ]:
# hent alle nødvendige kolonner
# kjør string trim på alle kolonner

konv = nin3_variabler[['Kortkode', 'NiN 2 kode', 'FP', 'SP']].copy()
konv = konv.drop(konv[(konv['NiN 2 kode'] == '') | (konv['NiN 2 kode'] == '-')].index) # remove rows where 'NiN 2 kode' is empty
konv = konv.drop(konv[(konv['Kortkode'] == '') | (konv['Kortkode'] == '-') |(konv['Kortkode'] == 'nan')].index) # remove rows where 'Kortkode' is empty
konv = konv.dropna(subset=['Kortkode'])
konv = konv.drop(konv[(konv['NiN 2 kode'] == 'nan')].index)
konv['NiN 2 kode'] = konv['NiN 2 kode'].str.replace('&', ',') # '&' means ',' so change to ','
konv['NiN 2 kode'] = konv['NiN 2 kode'].str.replace(' ', '') # remove whitespace in 'NiN 2 kode'
konv = konv.drop(konv[(konv['Kortkode'] == '') | (konv['Kortkode'] == '-') |(konv['Kortkode'] == 'nan')].index)
konv['SP'] = konv['SP'].str.replace('.0', '').str.replace('nan', '')
konv['FP'] = konv['SP'].str.replace('.0', '').str.replace('nan', '')
konv['NiN 2 kode'] = konv['NiN 2 kode'].str.replace('·', '-')
konv = konv.rename(columns={'NiN 2 kode': 'forrigekode'})
konv['Klasse'] = 'VN'# setter alle rader i kolonnen 'Klasse' til 'VN'
konv.to_csv('tmp/variabelnavn_konv_tmp.csv', index=False, sep=";")
#konv.info() 
# store to temp csv file

# FROM GT<>KONV example
gt_rows = []
with open('tmp/variabelnavn_konv_tmp.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    all_rows = []
    #check rows for multiple nin2koder
    for r in reader:
        if "," in r.get("forrigekode"):
            n2list = make_list_nin2kode(r.get("forrigekode"))
            for n2 in n2list:
                new_row = r.copy()
                new_row["forrigekode"] = n2.strip()
                all_rows.append(new_row)
    
    #loop over rows
    for row in all_rows:
        new = {} # new dict to store the new row
        new['VNKode']=row['Kortkode']
        new['FP']=row['FP']
        new['SP']=row['SP']
        new['Klasse']=row['Klasse']
        result = create_v23_variabel_url(row['forrigekode'])
        new['forrigekode']=result.get('kode23')
        new['url']=result.get('url')
        if not new.get('url'):# bytter siste "-" med "_" og prøver igjen
            kode = row['forrigekode'].rsplit('-', 1)
            kode = '_'.join(kode)
            result = create_v23_variabel_url(kode)
            new['forrigekode']=result.get('kode23')
            new['url']=result.get('url')
        # Sjekker om excel-kolonnen forrigekode (nin 2 koden) hadde match i koder23-dictonary 
        if not new.get('url'):
            new['forrigekode']=row['forrigekode'] 
            print(f"Fant ikke url for NiN2kode:{row['forrigekode']}")
        gt_rows.append(new)

# Creating konvertering csv for GT
with open('ut_data/konvertering_vn_v30.csv', 'w', newline='') as csvfile:
    fieldnames = ['VNKode','forrigekode','FP','SP','Klasse', 'url']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    writer.writerows(gt_rows)
    print(f"\n\nFile written to {csvfile.name}")    